# Setting up a DELFT3D simulation

This is the outline of the dset.py script from /src/ folder that sets up a complete DELFT3D simulation

In [1]:
#python modules
import numpy as np
import datetime
import sys
import os
from shutil import copy2
import glob
import os
import xml.dom.minidom as md
from tqdm import tqdm
import subprocess
import pickle


In [2]:
%load_ext pypath_magic

In [32]:
# add temporarily to the python the required local paths. Make sure you clean up afterwards.
# It won't run if ~/pypath_magic.pth is not empty to begin with.
%pypath -a ../src/
%pypath -a ../utils/

Added u'/home/critechuser/REPOS/StormS/src' to path.
Added u'/home/critechuser/REPOS/StormS/utils' to path.


In [33]:
#local modules
from meteo import wmap
from grid import *
from dep import *
from dem import readem
from idelft3d import meteo2delft3d
import mdf
from setobs import createf
from writenc import writenc
from bnd import tidebound

In [34]:
# clean up 
%pypath -d ../src/
%pypath -d ../utils/

Deleted u'/home/critechuser/REPOS/StormS/src' from path
Deleted u'/home/critechuser/REPOS/StormS/utils' from path


## input variables

In [6]:
# lat/lon window
lon0=-47.
lon1=44.
lat0=25.
lat1=76.

In [7]:
nt=12 # number of forecasting intervals (hours)

In [8]:
resolution = .5 # desired resolution in decimal degrees.

In [9]:
resmin=resolution * 60 #convert to arc minutes

There is an option to read a grid (lat lon) or create one. Below we choose the second option. See dset.py for more info.

In [10]:
#modify the lat/lon window to match the resolution 
ni=int((lon1-lon0)/resolution)+1
nj=int((lat1-lat0)/resolution)+1
  
lon1=lon0+(ni-1)*resolution
lat1=lat0+(nj-1)*resolution

In [11]:
basename='test' # the name tag of the simulation. It will used in all the required files --see below.

In [12]:
#This is the date stamp of the run. It will also be used the name of the simulation folder in YearMonthDay.Hour format.
runtime = datetime.datetime.strptime('12 Feb 2017 12:00', '%d %b %Y %H:%M')

In [13]:
path = '../test/' # the location where the simulation will be setup

In [14]:
# create the folder/run path
rpath=datetime.datetime.strftime(runtime,'%Y%m%d.%H' )

In [15]:
calc_dir=path+'{}/'.format(rpath)  #run subfolder 
if not os.path.exists(calc_dir): #create folder if it doesn't exist
      os.makedirs(calc_dir)


In [16]:
Tstart=60.*runtime.hour #minutes

Tstop=Tstart+60.*nt #minutes

In [17]:
'run attributes lons=[{}, {}], lats=[{}, {}], ni={}, nj={}, resolution={} decimal degrees ({} minutes)\n'.format(lon0,lon1,lat0,lat1,ni,nj,resolution,resmin)

'run attributes lons=[-47.0, 44.0], lats=[25.0, 76.0], ni=183, nj=103, resolution=0.5 decimal degrees (30.0 minutes)\n'

In [18]:
# save attributes in a python dictionary
dic={}
dic['bname']=basename
dic['lon0']=lon0
dic['lon1']=lon1
dic['lat0']=lat0
dic['lat1']=lat1
dic['ni']=ni
dic['nj']=nj
dic['nt']=nt

case=path.split('/')[-2]

# save dictionary in top folder
with open(path+case+'.pkl', 'w') as f:
    pickle.dump(dic,f)

## Create forcing

In [19]:
#get meteo for that day... see Meteo.ipynb for more info
p,u,v,elat,elon = wmap(runtime,0,3*(nt+1),lon0,lon1,lat0,lat1)

100%|██████████| 39/39 [01:56<00:00,  3.67s/it]      | 1/39 [00:05<03:17,  5.19s/it]


meteo done


In [20]:
# write variables in netcdf
t=np.arange(0,nt+1) # define time array

In [21]:
writenc(calc_dir+'/uvp.nc',elat[:,0],elon[0,:],u,v,p,t,rpath)

In [22]:
#write u,v,p files 
dlat=elat[1,0]-elat[0,0] #dlat required in meteo2delft
dlon=elon[0,1]-elon[0,0] #dlon required 
mlat0=elat[0,0] 
mlon0=elon[0,0] 


In [23]:
meteo2delft3d(p,u,v,lat0,lon0,dlat,dlon,runtime,nt,path=calc_dir,curvi=False) # write to ascii delft3d format

##  Get bathymetry interpolated onto lon,lat

In [24]:
# set the grid 
x=np.linspace(lon0,lon1,ni)
y=np.linspace(lat0,lat1,nj)
lon,lat=np.meshgrid(x,y)

In [25]:
pathb='../BATHYMETRY/dem.nc' #global bathymetry file

In [26]:
bat = readem(lat0,lat1,lon0,lon1,pathb,lon,lat,plot=False,interpolate=True)

In [27]:
bat = - bat # reverse for the hydro run
bat[bat<0]=-999. # mask all dry points

In [28]:
# Write bathymetry file
ba = Dep() # assign a bathymetry class variable
# append the line/column of nodata 
nodata=np.empty(ni)
nodata.fill(np.nan)
bat1=np.vstack((bat,nodata))
nodata=np.empty((nj+1,1))
nodata.fill(np.nan)
bat2=np.hstack((bat1,nodata))
ba.val = -bat2
ba.shape = bat2.shape

Dep.write(ba,calc_dir+basename+'.dep') #write dep file

## Create the GRID file

In [29]:
grd = Grid() #assign a grid class variable
  
grd.shape = bat.shape
grd.x = lon
grd.y = lat
grd.properties = {'Coordinate System': 'Spherical', 'alfori': 0.0, 'xori': 0.0, 'yori': 0.0}
  
grd.write(calc_dir+basename+'.grd') # write grd file

In [35]:
# Write bnd/bca file
tidebound(calc_dir,basename,grd,ba,10)



## Rest of the files

In [36]:
# Write .enc file
  
with open(calc_dir+basename+'.enc','w') as f:
    f.write('{:>5}{:>5}\n'.format(ni+1,1))  # add one like ddb. This is the fortran/python conversion 0->1
    f.write('{:>5}{:>5}\n'.format(ni+1,nj+1))
    f.write('{:>5}{:>5}\n'.format(1,nj+1))
    f.write('{:>5}{:>5}\n'.format(1,1))
    f.write('{:>5}{:>5}\n'.format(ni+1,1))
  

In [37]:
# Write .obs file. See Obs_points.ipynb
createf(calc_dir,basename,lat0,lat1,lon0,lon1,grd,ba)

In [38]:
# Define the mdf input file
# first read the default
inp, order = mdf.read('../src/default.mdf')

In [39]:
# modify default mdf file

#Set grid file
inp['Filcco']=basename+'.grd'
  
#Set enc file
inp['Filgrd']=basename+'.enc'
  
#Set dep file
inp['Fildep']=basename+'.dep'
  
#Set obs file
inp['Filsta']=basename+'.obs'
  
# adjust ni,nj
inp['MNKmax']=[ni+1,nj+1,1]  # add one like ddb
  
# adjust iteration date
inp['Itdate']=datetime.datetime.strftime(runtime.date(),'%Y-%m-%d')
  
#set time unit
inp['Tunit']='M'

#adjust iteration start
inp['Tstart']=[Tstart]
  
#adjust iteration stop
inp['Tstop']=[Tstop]
  
#adjust time step
inp['Dt']=[1.]
  
#adjust time for output
step=60
inp['Flmap']=[Tstart,step,Tstop]
inp['Flhis']=[Tstart,1,Tstop]
inp['Flpp']=[0,0,0]
inp['Flrst']=[720]
  
#time interval to smooth the hydrodynamic boundary conditions
inp['Tlfsmo']=[0.]

# specify ini file
# if 'Filic' not in order: order.append('Filic')
# inp['Filic']=basename+'.ini'

# netCDF output
if 'FlNcdf' not in order: order.append('FlNcdf')
inp['FlNcdf'] = 'map his'

  
#SAVE mdf file
mdf.write(inp, calc_dir+basename+'.mdf',selection=order)


In [40]:
# edit and save config files
#DELFT3D config xml file
copy2('../src/config_d_hydro.xml',calc_dir+'config_d_hydro.xml')

xml=md.parse(calc_dir+'config_d_hydro.xml')

xml.getElementsByTagName('mdfFile')[0].firstChild.replaceWholeText(basename+'.mdf')

with open(calc_dir+'config_d_hydro.xml','w') as f:
      xml.writexml(f)

#execution shell script        
copy2('../src/run_flow2d3d.sh',calc_dir+'run_flow2d3d.sh')

## Testing the configuration - Run 

In [41]:
# note that cwd is the folder where the executable is
ex=subprocess.Popen('./run_flow2d3d.sh', cwd=calc_dir, shell=True, stdout=subprocess.PIPE, bufsize=1)
for line in iter(ex.stdout.readline,b''): print line
ex.stdout.close()

MPI process number 001 has host unknown and is running on processor POSEIDON

MPI process number 002 has host unknown and is running on processor POSEIDON

MPI process number 003 has host unknown and is running on processor POSEIDON

MPI process number 000 has host unknown and is running on processor POSEIDON

--------------------------------------------------------------------------------

       Deltares, FLOW2D3D Version 6.02.05.6131, May 17 2016, 12:19:14

       libflow2d3d.so entry Flow2D3D::Run

--------------------------------------------------------------------------------



Part I    - Initialisation Time Dep. Data module...    

            runid : test

Part II   - Creating intermediate files...             

Part III  - Initialisation of the Execution module...  

Part IV   - Reading complete MD-file...                     

Part V    - Initialisation & checking input...              

Part VI   - Initialisation & checking second part...        

Part VII  - Initialisatio